In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import os
import pandas as pd
from scipy import stats
from pydub import AudioSegment
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav



In [2]:
df = pd.read_csv("cv-valid-train.csv", nrows=2000)

In [3]:
df

,Unnamed: 0,filename,age,gender,accent
0,5,cv-valid-train/sample-000005.wav,20s,female,us
1,8,cv-valid-train/sample-000008.wav,70s,male,us
2,13,cv-valid-train/sample-000013.wav,30s,female,us
3,14,cv-valid-train/sample-000014.wav,60s,male,england
4,19,cv-valid-train/sample-000019.wav,50s,male,australia
...,...,...,...,...,...
1995,6234,cv-valid-train/sample-006234.wav,40s,male,us
1996,6236,cv-valid-train/sample-006236.wav,10s,female,england
1997,6238,cv-valid-train/sample-006238.wav,40s,male,england
1998,6247,cv-valid-train/sample-006247.wav,50s,male,us


In [4]:
df2=df[['filename','gender']]
df3=df2.dropna(axis=0,how='any')

In [5]:
df3

,filename,gender
0,cv-valid-train/sample-000005.wav,female
1,cv-valid-train/sample-000008.wav,male
2,cv-valid-train/sample-000013.wav,female
3,cv-valid-train/sample-000014.wav,male
4,cv-valid-train/sample-000019.wav,male
...,...,...
1995,cv-valid-train/sample-006234.wav,male
1996,cv-valid-train/sample-006236.wav,female
1997,cv-valid-train/sample-006238.wav,male
1998,cv-valid-train/sample-006247.wav,male


In [6]:
df4=df3
df4['filename']=df3['filename'].replace('wav','npz',regex=True)
df4['filename']=df4['filename'].replace('cv-valid-train','cv-valid-train-fft',regex=True)
df4

,filename,gender
0,cv-valid-train-fft/sample-000005.npz,female
1,cv-valid-train-fft/sample-000008.npz,male
2,cv-valid-train-fft/sample-000013.npz,female
3,cv-valid-train-fft/sample-000014.npz,male
4,cv-valid-train-fft/sample-000019.npz,male
...,...,...
1995,cv-valid-train-fft/sample-006234.npz,male
1996,cv-valid-train-fft/sample-006236.npz,female
1997,cv-valid-train-fft/sample-006238.npz,male
1998,cv-valid-train-fft/sample-006247.npz,male


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df4['gender'])
list(le.classes_)
le.transform(df4['gender'])
df4['gender']=le.transform(df4['gender'])

In [8]:
df4

,filename,gender
0,cv-valid-train-fft/sample-000005.npz,0
1,cv-valid-train-fft/sample-000008.npz,1
2,cv-valid-train-fft/sample-000013.npz,0
3,cv-valid-train-fft/sample-000014.npz,1
4,cv-valid-train-fft/sample-000019.npz,1
...,...,...
1995,cv-valid-train-fft/sample-006234.npz,1
1996,cv-valid-train-fft/sample-006236.npz,0
1997,cv-valid-train-fft/sample-006238.npz,1
1998,cv-valid-train-fft/sample-006247.npz,1


In [9]:
data = np.load("cv-valid-train-fft/sample-000005.npz")
data["fourier"].shape

(1750,)

In [10]:
data = np.load("cv-valid-train-fft/sample-000008.npz")
data["fourier"].shape

(519,)

In [11]:
def eq_sampling(vec,n):
    tot = vec.shape[0]
    j = 0
    out = np.zeros(n)
    for i in range(n):
        j=int(np.floor(i/n*tot))+1 
        out[i]=vec[j]
    return out
        

In [12]:
data["fourier"][176]

(1.4214624181858084-0.7599706144254148j)

In [13]:
feature_dim=100

In [14]:
new_dt= np.zeros((2000,feature_dim))
i=0
for x in df4['filename']:
    dt= np.load(x)
    new_dt[i]=eq_sampling(dt["fourier"],feature_dim)
    i=i+1

In [15]:
pd.DataFrame(new_dt)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-4.718541,-7.431879,-9.909642,-7.747231,-7.069311,-4.214984,-10.301658,-9.594640,-13.774135,-11.329517,...,0.242505,-3.786584,-7.632050,0.172917,-2.671390,-2.763424,-2.999502,-1.143826,-0.562238,-0.850744
1,-8.171611,-8.630433,-11.379080,-14.037780,-7.048682,-6.472766,-5.723874,-4.661712,-5.525408,-4.826827,...,6.247595,4.290538,8.887937,5.153458,2.350927,0.515604,7.551008,4.282878,5.043225,6.956236
2,-5.281976,-11.223109,-9.685950,-6.384448,-8.991645,-3.473988,-6.049644,-4.828456,-12.864729,-9.141106,...,0.259432,5.722704,11.124176,11.953588,11.346116,10.223483,7.749689,8.735780,3.313072,2.874254
3,-2.489021,-1.329008,1.708029,-15.127816,-1.403231,-2.492549,-3.666792,-7.598074,0.583617,-1.780904,...,-3.451815,8.578359,8.897879,9.665276,8.233952,9.400926,9.495286,8.458951,11.976034,13.431938
4,-2.801097,0.571298,-7.651868,3.855909,7.948016,0.443122,8.444209,10.942819,11.443191,10.208493,...,11.292989,12.688521,7.950798,10.215882,9.510152,9.254118,10.788183,10.986886,12.005495,13.994588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-1.101538,1.120907,1.178763,-0.911027,-2.225496,-1.463268,-1.577780,-3.639596,0.493375,0.859682,...,11.513390,11.651777,8.200988,9.890290,10.225504,11.171914,11.081766,11.656277,13.681867,15.625271
1996,3.722241,-5.535346,2.462404,1.613422,-6.419888,-0.295587,3.759354,-0.055165,-1.924295,-0.264832,...,8.589615,7.725631,1.233177,-4.645425,2.546700,3.088415,-0.519331,4.288110,11.075745,2.862231
1997,-0.556666,-2.218499,-2.742744,-9.753425,-5.995477,-5.371433,-6.682977,-7.608422,-4.514638,-11.453700,...,6.670831,3.418897,3.477213,0.771008,-1.282365,4.226561,3.658335,2.989869,2.613826,2.361913
1998,-3.303773,-3.442440,-5.825139,-4.867700,-6.450498,-12.569829,-8.351196,-3.918032,-13.010506,-2.751451,...,11.489163,10.078828,9.825076,8.279032,7.265529,8.423137,7.003515,1.924860,6.557036,13.192730


In [16]:
df_ready=pd.concat([df4,pd.DataFrame(new_dt)],axis=1)

In [17]:
df_ready

,filename,gender,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,cv-valid-train-fft/sample-000005.npz,0,-4.718541,-7.431879,-9.909642,-7.747231,-7.069311,-4.214984,-10.301658,-9.594640,...,0.242505,-3.786584,-7.632050,0.172917,-2.671390,-2.763424,-2.999502,-1.143826,-0.562238,-0.850744
1,cv-valid-train-fft/sample-000008.npz,1,-8.171611,-8.630433,-11.379080,-14.037780,-7.048682,-6.472766,-5.723874,-4.661712,...,6.247595,4.290538,8.887937,5.153458,2.350927,0.515604,7.551008,4.282878,5.043225,6.956236
2,cv-valid-train-fft/sample-000013.npz,0,-5.281976,-11.223109,-9.685950,-6.384448,-8.991645,-3.473988,-6.049644,-4.828456,...,0.259432,5.722704,11.124176,11.953588,11.346116,10.223483,7.749689,8.735780,3.313072,2.874254
3,cv-valid-train-fft/sample-000014.npz,1,-2.489021,-1.329008,1.708029,-15.127816,-1.403231,-2.492549,-3.666792,-7.598074,...,-3.451815,8.578359,8.897879,9.665276,8.233952,9.400926,9.495286,8.458951,11.976034,13.431938
4,cv-valid-train-fft/sample-000019.npz,1,-2.801097,0.571298,-7.651868,3.855909,7.948016,0.443122,8.444209,10.942819,...,11.292989,12.688521,7.950798,10.215882,9.510152,9.254118,10.788183,10.986886,12.005495,13.994588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,cv-valid-train-fft/sample-006234.npz,1,-1.101538,1.120907,1.178763,-0.911027,-2.225496,-1.463268,-1.577780,-3.639596,...,11.513390,11.651777,8.200988,9.890290,10.225504,11.171914,11.081766,11.656277,13.681867,15.625271
1996,cv-valid-train-fft/sample-006236.npz,0,3.722241,-5.535346,2.462404,1.613422,-6.419888,-0.295587,3.759354,-0.055165,...,8.589615,7.725631,1.233177,-4.645425,2.546700,3.088415,-0.519331,4.288110,11.075745,2.862231
1997,cv-valid-train-fft/sample-006238.npz,1,-0.556666,-2.218499,-2.742744,-9.753425,-5.995477,-5.371433,-6.682977,-7.608422,...,6.670831,3.418897,3.477213,0.771008,-1.282365,4.226561,3.658335,2.989869,2.613826,2.361913
1998,cv-valid-train-fft/sample-006247.npz,1,-3.303773,-3.442440,-5.825139,-4.867700,-6.450498,-12.569829,-8.351196,-3.918032,...,11.489163,10.078828,9.825076,8.279032,7.265529,8.423137,7.003515,1.924860,6.557036,13.192730


In [18]:
from sklearn.model_selection import train_test_split
X=df_ready.iloc[:,2:(2+feature_dim-1)]
y=df_ready['gender']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 78,  26],
       [ 15, 281]], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8975

In [21]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.9320066334991708

In [44]:
df2=df[['filename','age']]
df3=df2.dropna(axis=0,how='any')
df4=df3
df4['filename']=df3['filename'].replace('wav','npz',regex=True)
df4['filename']=df4['filename'].replace('cv-valid-train','cv-valid-train-fft',regex=True)
df4

,filename,age
0,cv-valid-train-fft/sample-000005.npz,20s
1,cv-valid-train-fft/sample-000008.npz,70s
2,cv-valid-train-fft/sample-000013.npz,30s
3,cv-valid-train-fft/sample-000014.npz,60s
4,cv-valid-train-fft/sample-000019.npz,50s
...,...,...
1995,cv-valid-train-fft/sample-006234.npz,40s
1996,cv-valid-train-fft/sample-006236.npz,10s
1997,cv-valid-train-fft/sample-006238.npz,40s
1998,cv-valid-train-fft/sample-006247.npz,50s


In [45]:
df4.loc[df4['age'] == '10s', 'age'] = 'Young'
df4.loc[df4['age'] == '20s', 'age'] = 'Young'
df4.loc[df4['age'] == '30s', 'age'] = 'Young'
df4.loc[df4['age'] == '40s', 'age'] = 'Old'
df4.loc[df4['age'] == '50s', 'age'] = 'Old'
df4.loc[df4['age'] == '60s', 'age'] = 'Old'
df4.loc[df4['age'] == '70s', 'age'] = 'Old'
df4.loc[df4['age'] == '80s', 'age'] = 'Old'

In [46]:
df4['age'].value_counts()

Young    1194
Old       806
Name: age, dtype: int64

In [47]:
df4

,filename,age
0,cv-valid-train-fft/sample-000005.npz,Young
1,cv-valid-train-fft/sample-000008.npz,Old
2,cv-valid-train-fft/sample-000013.npz,Young
3,cv-valid-train-fft/sample-000014.npz,Old
4,cv-valid-train-fft/sample-000019.npz,Old
...,...,...
1995,cv-valid-train-fft/sample-006234.npz,Old
1996,cv-valid-train-fft/sample-006236.npz,Young
1997,cv-valid-train-fft/sample-006238.npz,Old
1998,cv-valid-train-fft/sample-006247.npz,Old


In [48]:
df_ready=pd.concat([df4,pd.DataFrame(new_dt)],axis=1)
df_ready

,filename,age,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,cv-valid-train-fft/sample-000005.npz,Young,-4.718541,-7.431879,-9.909642,-7.747231,-7.069311,-4.214984,-10.301658,-9.594640,...,0.242505,-3.786584,-7.632050,0.172917,-2.671390,-2.763424,-2.999502,-1.143826,-0.562238,-0.850744
1,cv-valid-train-fft/sample-000008.npz,Old,-8.171611,-8.630433,-11.379080,-14.037780,-7.048682,-6.472766,-5.723874,-4.661712,...,6.247595,4.290538,8.887937,5.153458,2.350927,0.515604,7.551008,4.282878,5.043225,6.956236
2,cv-valid-train-fft/sample-000013.npz,Young,-5.281976,-11.223109,-9.685950,-6.384448,-8.991645,-3.473988,-6.049644,-4.828456,...,0.259432,5.722704,11.124176,11.953588,11.346116,10.223483,7.749689,8.735780,3.313072,2.874254
3,cv-valid-train-fft/sample-000014.npz,Old,-2.489021,-1.329008,1.708029,-15.127816,-1.403231,-2.492549,-3.666792,-7.598074,...,-3.451815,8.578359,8.897879,9.665276,8.233952,9.400926,9.495286,8.458951,11.976034,13.431938
4,cv-valid-train-fft/sample-000019.npz,Old,-2.801097,0.571298,-7.651868,3.855909,7.948016,0.443122,8.444209,10.942819,...,11.292989,12.688521,7.950798,10.215882,9.510152,9.254118,10.788183,10.986886,12.005495,13.994588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,cv-valid-train-fft/sample-006234.npz,Old,-1.101538,1.120907,1.178763,-0.911027,-2.225496,-1.463268,-1.577780,-3.639596,...,11.513390,11.651777,8.200988,9.890290,10.225504,11.171914,11.081766,11.656277,13.681867,15.625271
1996,cv-valid-train-fft/sample-006236.npz,Young,3.722241,-5.535346,2.462404,1.613422,-6.419888,-0.295587,3.759354,-0.055165,...,8.589615,7.725631,1.233177,-4.645425,2.546700,3.088415,-0.519331,4.288110,11.075745,2.862231
1997,cv-valid-train-fft/sample-006238.npz,Old,-0.556666,-2.218499,-2.742744,-9.753425,-5.995477,-5.371433,-6.682977,-7.608422,...,6.670831,3.418897,3.477213,0.771008,-1.282365,4.226561,3.658335,2.989869,2.613826,2.361913
1998,cv-valid-train-fft/sample-006247.npz,Old,-3.303773,-3.442440,-5.825139,-4.867700,-6.450498,-12.569829,-8.351196,-3.918032,...,11.489163,10.078828,9.825076,8.279032,7.265529,8.423137,7.003515,1.924860,6.557036,13.192730


In [52]:
from sklearn.model_selection import train_test_split
X=df_ready.iloc[:,2:(2+feature_dim-1)]
y=df_ready['age']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

from sklearn import svm
clf = svm.SVC(kernel='rbf') # Linear Kernel
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6175

In [54]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 32, 126],
       [ 27, 215]], dtype=int64)

In [55]:
df2=df[['filename','accent']]
df3=df2.dropna(axis=0,how='any')
df4=df3
df4['filename']=df3['filename'].replace('wav','npz',regex=True)
df4['filename']=df4['filename'].replace('cv-valid-train','cv-valid-train-fft',regex=True)
df4

,filename,accent
0,cv-valid-train-fft/sample-000005.npz,us
1,cv-valid-train-fft/sample-000008.npz,us
2,cv-valid-train-fft/sample-000013.npz,us
3,cv-valid-train-fft/sample-000014.npz,england
4,cv-valid-train-fft/sample-000019.npz,australia
...,...,...
1995,cv-valid-train-fft/sample-006234.npz,us
1996,cv-valid-train-fft/sample-006236.npz,england
1997,cv-valid-train-fft/sample-006238.npz,england
1998,cv-valid-train-fft/sample-006247.npz,us


In [56]:
df4.loc[df4['accent'] == 'us', 'accent'] = 'us'
df4.loc[df4['accent'] != 'us', 'accent'] = 'non-us'

In [57]:
df4['accent'].value_counts()

non-us    1076
us         924
Name: accent, dtype: int64

In [58]:
df_ready=pd.concat([df4,pd.DataFrame(new_dt)],axis=1)
df_ready

,filename,accent,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,cv-valid-train-fft/sample-000005.npz,us,-4.718541,-7.431879,-9.909642,-7.747231,-7.069311,-4.214984,-10.301658,-9.594640,...,0.242505,-3.786584,-7.632050,0.172917,-2.671390,-2.763424,-2.999502,-1.143826,-0.562238,-0.850744
1,cv-valid-train-fft/sample-000008.npz,us,-8.171611,-8.630433,-11.379080,-14.037780,-7.048682,-6.472766,-5.723874,-4.661712,...,6.247595,4.290538,8.887937,5.153458,2.350927,0.515604,7.551008,4.282878,5.043225,6.956236
2,cv-valid-train-fft/sample-000013.npz,us,-5.281976,-11.223109,-9.685950,-6.384448,-8.991645,-3.473988,-6.049644,-4.828456,...,0.259432,5.722704,11.124176,11.953588,11.346116,10.223483,7.749689,8.735780,3.313072,2.874254
3,cv-valid-train-fft/sample-000014.npz,non-us,-2.489021,-1.329008,1.708029,-15.127816,-1.403231,-2.492549,-3.666792,-7.598074,...,-3.451815,8.578359,8.897879,9.665276,8.233952,9.400926,9.495286,8.458951,11.976034,13.431938
4,cv-valid-train-fft/sample-000019.npz,non-us,-2.801097,0.571298,-7.651868,3.855909,7.948016,0.443122,8.444209,10.942819,...,11.292989,12.688521,7.950798,10.215882,9.510152,9.254118,10.788183,10.986886,12.005495,13.994588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,cv-valid-train-fft/sample-006234.npz,us,-1.101538,1.120907,1.178763,-0.911027,-2.225496,-1.463268,-1.577780,-3.639596,...,11.513390,11.651777,8.200988,9.890290,10.225504,11.171914,11.081766,11.656277,13.681867,15.625271
1996,cv-valid-train-fft/sample-006236.npz,non-us,3.722241,-5.535346,2.462404,1.613422,-6.419888,-0.295587,3.759354,-0.055165,...,8.589615,7.725631,1.233177,-4.645425,2.546700,3.088415,-0.519331,4.288110,11.075745,2.862231
1997,cv-valid-train-fft/sample-006238.npz,non-us,-0.556666,-2.218499,-2.742744,-9.753425,-5.995477,-5.371433,-6.682977,-7.608422,...,6.670831,3.418897,3.477213,0.771008,-1.282365,4.226561,3.658335,2.989869,2.613826,2.361913
1998,cv-valid-train-fft/sample-006247.npz,us,-3.303773,-3.442440,-5.825139,-4.867700,-6.450498,-12.569829,-8.351196,-3.918032,...,11.489163,10.078828,9.825076,8.279032,7.265529,8.423137,7.003515,1.924860,6.557036,13.192730


In [59]:
from sklearn.model_selection import train_test_split
X=df_ready.iloc[:,2:(2+feature_dim-1)]
y=df_ready['accent']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

from sklearn import svm
clf = svm.SVC(kernel='rbf') # Linear Kernel
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5625

In [61]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred), end="") 

[[167  42]
 [133  58]]